# 회귀 ; regression
- 딥러닝 ==> 매우 많은 1차 방정식의 모음

In [1]:
import torch
import torch.nn as nn          # 신경망을 만드는 클래스
import torch.optim as optim    # 학습을 원활하게 도와주는 도구
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'현재 사용 중인 Device : {device}')

현재 사용 중인 Device : cuda


In [3]:
# 데이터 가져오기
tips = sns.load_dataset('tips').dropna()     # 결측치 제거
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
# 데이터 분리
x = tips[['total_bill', 'size']].values      # 두 변수 가져오기   
## 딥러닝은 기본적으로 행렬 연산임
## padnas는 사이킬런과 다르게 ~~
## numpy 배열 형태로 바꿔줘야함.
## ????????????????????????????????????????????
y = tips['tip'].values.astype(np.float32)    # 회귀 대상

In [5]:
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2,
    random_state=42
)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((195, 2), (49, 2), (195,), (49,))

In [6]:
# 스케일링 -> 필수 아님. 각 변수의 성격을 잘 봐야함
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [7]:
# Tensor 변환 : PyTorch 프레임워크에 입력값으로 내가 추가를 하겠다. 형변환 필수
# 이때 형변환 진행할 때, device 같이 입력
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

In [8]:
# 회귀 모형 정의
# 텐서플로에서는 입문자들이 굳이 클래스를 만들 필요 없음
# 파이토치는 무조건 클래스를 만들어야 함. -> 문법의 차이 (텐서 vs 파이토치)

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(2,1)                   # 2:독립변수의 개수, 1: 출력 개수 -> 수치예측이니깐 1개만 출력함

    def forward(self, x):
        return self.linear(x)                          # 선형변환

In [9]:
# 모델 초기화
model = RegressionModel().to(device)                   # GPU를 쓰면서 모델을 쓰겠다. -> GPU 회귀 모형 사용
# 아래 두 개념은 경사하강법에서 출발 (MSELoss, Adam)
# ML의 예시 : LightGBM, XGBoost, CatBoost
criterion = nn.MSELoss()                               # 평균 제곱 오차 손실 함수 (Loss Function)
optimizer = optim.Adam(model.parameters(), lr=0.01)    # Adam 최적화 알고리즘

In [10]:
# 모델 학습
epochs = 1000                                           # 보통 epochs이라는 변수를 씀
for epoch in range(epochs) :
    model.train()                                       # 학습 모드
    optimizer.zero_grad()                               # 그래디언트 초기화 (학습하고 초기화 또 학습하고 초기화)
    outputs = model(x_train_tensor)                     # 순전파
    loss = criterion(outputs, y_train_tensor)           # 평가지표 계산
    loss.backward()                                     # 역전파
    optimizer.step()                                    # 파라미터 (기울기,절편) 업데이트, 행렬로 저장

    # 100 에포크마다 진행상황 출력
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [100/1000], Loss: 8.6119
Epoch [200/1000], Loss: 4.8619
Epoch [300/1000], Loss: 2.7924
Epoch [400/1000], Loss: 1.7782
Epoch [500/1000], Loss: 1.3438
Epoch [600/1000], Loss: 1.1830
Epoch [700/1000], Loss: 1.1319
Epoch [800/1000], Loss: 1.1181
Epoch [900/1000], Loss: 1.1149
Epoch [1000/1000], Loss: 1.1142


# 분류

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [47]:
# 디바이스 설정
device = torch .device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [48]:
# 데이터 가져오기
tips = sns.load_dataset('tips').dropna()
tips.head(1)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2


In [49]:
# 특성, 타겟 설정
x = tips[['total_bill', 'tip', 'size']].values
le = LabelEncoder()
y = le.fit_transform(tips['day'])                # 입력값을 입력하면 요일을 맞추겠다..
y                                                # Thur = 3, Fri = 0, sat = 2, sun = 1

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3])

In [50]:
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2,
    random_state=42
)

In [51]:
# 스케일링
scaler = StandardScaler()
x_trian = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [52]:
# Tensor 변환 
# y값에 대해서만 long값으로 변경 + 언스퀴지도 삭제
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

In [56]:
# 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.linear = nn.Linear(3,4)                # 위에 수치예측에선 (2,1)로 정의함 (입력2, 출력1)
                                                    # 여기는 입력변수 3개, 출력값 4개로 설정함
                                                    # 왜?
    def forward(self, x): 
        return self.linear(x)                   # CrossEntropyLoss 활용, softmax 자동 계산됨

In [57]:
# 모델 초기화
model = ClassificationModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [58]:
# 학습
epochs = 300  # 총 300번 학습을 돌림
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(x_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [50/300], Loss: 1.3999
Epoch [100/300], Loss: 1.2980
Epoch [150/300], Loss: 1.2659
Epoch [200/300], Loss: 1.2526
Epoch [250/300], Loss: 1.2457
Epoch [300/300], Loss: 1.2421


In [61]:
# 평가
model.eval()
with torch.no_grad():
    outputs = model(x_test_tensor)
    _, predicted = torch.max(outputs,1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0)
    print(accuracy)

0.30612244897959184
